# Placing an order

## Check for time consistency with server

In [1]:
import torch
torch.cuda.is_available() 

True

In [2]:
import requests
import time
# check for condition supplied in guide: 
# https://bybit-exchange.github.io/docs/v5/guide

'''
Please make sure that the timestamp parameter adheres to the following rule:
server_time - recv_window <= timestamp < server_time + 1000
server_time stands for Bybit server time, which can be queried via the Server Time endpoint.   
'''

url = "https://api-testnet.bybit.com/v5/market/time"
req = requests.request("GET", url)
res = req.json()
print(res["time"])#.result.time)
print(int(time.time() * 1000))
print(res["time"]-5000 <= int(time.time() * 1000) and int(time.time() * 1000) < res["time"])

1717678366048
1717678366145
False


# directly from bybit examples

## important info
It seems that there are four server spaces at bybit: mainnet, testnet, demo and demo-testnet. The Demo space seems to be the most fitting for the purpose I intend to use it, namely letting the bot run a little. See for further explanation: https://www.bybit.com/en/help-center/article/FAQ-Demo-Trading


**other infos**

get acc balance: https://bybit-exchange.github.io/docs/api-explorer/v5/account/wallet

integration guide: https://bybit-exchange.github.io/docs/v5/guide

demo von dem untenstehendem code: https://github.com/bybit-exchange/api-usage-examples/blob/master/V5_demo/api_demo/Encryption_HMAC.py

liste von bybit errors: https://bybit-exchange.github.io/docs/v5/error

In [3]:
import requests
import time
import hashlib
import hmac
import uuid

# HMAC KEY FOR TESTNET > idk NET
#api_key = "vgcrg9nWw7owVYEDdc"
#secret_key = "YPz7eb0YzhOpa9s2yMEj3P4UdXrmSEtqRDCP"

# TESTNET
# HMAC KEY FOR READ ONLY WALLET BALANCE
# api_key = "kO4Wf9sdmWk0IeFRBc"
# secret_key = "ZxyPUKE893Alxq7aFdNSZUo5xYGrcp5hKgNO"

# DEMO NET
api_key = "GMUtVbMzRhohkOagoW"
secret_key = "vexur1jjKy5AiQE0rXMhVYbjRFRy6jVxhIhf"

httpClient=requests.Session()
recv_window=str(10000)
# url = "https://api-testnet.bybit.com" # Testnet endpoint
url = "https://api-demo.bybit.com" # DEMO endpoint

def HTTP_Request(endPoint,method,payload,Info):
    global time_stamp
    time_stamp=str(int(time.time() * 10 ** 3))
    signature=genSignature(payload)
    headers = {
        'X-BAPI-API-KEY': api_key,
        'X-BAPI-SIGN': signature,
        'X-BAPI-SIGN-TYPE': '2',
        'X-BAPI-TIMESTAMP': time_stamp,
        'X-BAPI-RECV-WINDOW': recv_window,
        'Content-Type': 'application/json'
    }
    if(method=="POST"):
        response = httpClient.request(method, url+endPoint, headers=headers, data=payload)
    else:
        response = httpClient.request(method, url+endPoint+"?"+payload, headers=headers)
    print(response)
    print(response.text)
    print(Info + " Elapsed Time : " + str(response.elapsed))
    return response#.json()

def genSignature(payload):
    param_str= str(time_stamp) + api_key + recv_window + payload
    hash = hmac.new(bytes(secret_key, "utf-8"), param_str.encode("utf-8"),hashlib.sha256)
    signature = hash.hexdigest()
    return signature

In [4]:
print(api_key)
#Get Wallet Balance
endpoint="/v5/account/wallet-balance"
method="GET"
params='accountType=UNIFIED'
res = HTTP_Request(endpoint,method,params,"UNIFIED")
res.json()


GMUtVbMzRhohkOagoW
<Response [200]>
{"retCode":0,"retMsg":"OK","result":{"list":[{"totalEquity":"174873.48165828","accountIMRate":"0","totalMarginBalance":"99985.95794728","totalInitialMargin":"7.13945405","accountType":"UNIFIED","totalAvailableBalance":"99978.81849322","accountMMRate":"0","totalPerpUPL":"-0.1190651","totalWalletBalance":"99986.07701238","accountLTV":"0","totalMaintenanceMargin":"0.39775995","coin":[{"availableToBorrow":"","bonus":"0","accruedInterest":"0","availableToWithdraw":"50000","totalOrderIM":"0","equity":"50000","totalPositionMM":"0","usdValue":"50000.85","unrealisedPnl":"0","collateralSwitch":true,"spotHedgingQty":"0","borrowAmount":"0.000000000000000000","totalPositionIM":"0","walletBalance":"50000","cumRealisedPnl":"0","locked":"0","marginCollateral":true,"coin":"USDC"},{"availableToBorrow":"","bonus":"0","accruedInterest":"0","availableToWithdraw":"1","totalOrderIM":"0","equity":"1","totalPositionMM":"0","usdValue":"71046.141743","unrealisedPnl":"0","colla

{'retCode': 0,
 'retMsg': 'OK',
 'result': {'list': [{'totalEquity': '174873.48165828',
    'accountIMRate': '0',
    'totalMarginBalance': '99985.95794728',
    'totalInitialMargin': '7.13945405',
    'accountType': 'UNIFIED',
    'totalAvailableBalance': '99978.81849322',
    'accountMMRate': '0',
    'totalPerpUPL': '-0.1190651',
    'totalWalletBalance': '99986.07701238',
    'accountLTV': '0',
    'totalMaintenanceMargin': '0.39775995',
    'coin': [{'availableToBorrow': '',
      'bonus': '0',
      'accruedInterest': '0',
      'availableToWithdraw': '50000',
      'totalOrderIM': '0',
      'equity': '50000',
      'totalPositionMM': '0',
      'usdValue': '50000.85',
      'unrealisedPnl': '0',
      'collateralSwitch': True,
      'spotHedgingQty': '0',
      'borrowAmount': '0.000000000000000000',
      'totalPositionIM': '0',
      'walletBalance': '50000',
      'cumRealisedPnl': '0',
      'locked': '0',
      'marginCollateral': True,
      'coin': 'USDC'},
     {'availa

In [5]:
'''
#Create Order
endpoint="/v5/order/create"
method="POST"
orderLinkId=uuid.uuid4().hex
params='{"category":"linear","symbol": "BTCUSDT","side": "Buy","positionIdx": 0,"orderType": "Limit","qty": "0.001","price": "60000","timeInForce": "GTC","orderLinkId": "' + orderLinkId + '"}'
HTTP_Request(endpoint,method,params,"Create")
'''

'\n#Create Order\nendpoint="/v5/order/create"\nmethod="POST"\norderLinkId=uuid.uuid4().hex\nparams=\'{"category":"linear","symbol": "BTCUSDT","side": "Buy","positionIdx": 0,"orderType": "Limit","qty": "0.001","price": "60000","timeInForce": "GTC","orderLinkId": "\' + orderLinkId + \'"}\'\nHTTP_Request(endpoint,method,params,"Create")\n'

In [28]:
#Create Order
import json
symbol = "BTCUSDT"
params={
"category":"linear",
"symbol": symbol,
"side": "Buy",
"orderType": "Limit",
"qty": "0.001",
"price": "60000"}

params = json.dumps(params)

#"positionIdx": 0,
#"timeInForce": "GTC"
#"orderLinkId": "' + orderLinkId + '"
endpoint="/v5/order/create"
method="POST"
orderLinkId=uuid.uuid4().hex
res = HTTP_Request(endpoint,method,params,"Create")
res

<Response [200]>
{"retCode":0,"retMsg":"OK","result":{"orderId":"b435fcdf-02c8-43f5-bd64-fcb8e64582b1","orderLinkId":""},"retExtInfo":{},"time":1717681928964}
Create Elapsed Time : 0:00:00.580417


<Response [200]>

In [7]:
# geet open orders // see "openOnly" option
endpoint="/v5/order/realtime"
method="GET"
params='category=linear&settleCoin=USDT'
res = HTTP_Request(endpoint,method,params,"Order List")
res = res.json()
num_orders = len(res['result']['list'])
symbols = [entry["symbol"] for entry in res['result']['list']]
res


<Response [200]>
{"retCode":0,"retMsg":"OK","result":{"nextPageCursor":"3c06db75-16e0-456e-9a02-6df05f64250a%3A1717678367486%2C2d0d5018-2c56-4606-a7cd-504f76080d89%3A1717673992329","category":"linear","list":[{"symbol":"BTCUSDT","orderType":"Limit","orderLinkId":"","slLimitPrice":"0","orderId":"3c06db75-16e0-456e-9a02-6df05f64250a","cancelType":"UNKNOWN","avgPrice":"","stopOrderType":"","lastPriceOnCreated":"71129.8","orderStatus":"New","createType":"CreateByUser","takeProfit":"","cumExecValue":"0","tpslMode":"","smpType":"None","triggerDirection":0,"blockTradeId":"","isLeverage":"","rejectReason":"EC_NoError","price":"60000","orderIv":"","createdTime":"1717678367486","tpTriggerBy":"","positionIdx":0,"timeInForce":"GTC","leavesValue":"60","updatedTime":"1717678367486","side":"Buy","smpGroup":0,"triggerPrice":"","tpLimitPrice":"0","cumExecFee":"0","leavesQty":"0.001","slTriggerBy":"","closeOnTrigger":false,"placeType":"","cumExecQty":"0","reduceOnly":false,"qty":"0.001","stopLoss":"","m

{'retCode': 0,
 'retMsg': 'OK',
 'result': {'nextPageCursor': '3c06db75-16e0-456e-9a02-6df05f64250a%3A1717678367486%2C2d0d5018-2c56-4606-a7cd-504f76080d89%3A1717673992329',
  'category': 'linear',
  'list': [{'symbol': 'BTCUSDT',
    'orderType': 'Limit',
    'orderLinkId': '',
    'slLimitPrice': '0',
    'orderId': '3c06db75-16e0-456e-9a02-6df05f64250a',
    'cancelType': 'UNKNOWN',
    'avgPrice': '',
    'stopOrderType': '',
    'lastPriceOnCreated': '71129.8',
    'orderStatus': 'New',
    'createType': 'CreateByUser',
    'takeProfit': '',
    'cumExecValue': '0',
    'tpslMode': '',
    'smpType': 'None',
    'triggerDirection': 0,
    'blockTradeId': '',
    'isLeverage': '',
    'rejectReason': 'EC_NoError',
    'price': '60000',
    'orderIv': '',
    'createdTime': '1717678367486',
    'tpTriggerBy': '',
    'positionIdx': 0,
    'timeInForce': 'GTC',
    'leavesValue': '60',
    'updatedTime': '1717678367486',
    'side': 'Buy',
    'smpGroup': 0,
    'triggerPrice': '',
  

In [8]:
#Get open positions
endpoint="/v5/position/list"
method="GET"
params='category=linear&settleCoin=USDT'
res = HTTP_Request(endpoint,method,params,"Position List")
res = res.json()
res['result']['list']

<Response [200]>
{"retCode":0,"retMsg":"OK","result":{"nextPageCursor":"BTCUSDT%2C1717669802036%2C0","category":"linear","list":[{"symbol":"BTCUSDT","leverage":"10","autoAddMargin":0,"avgPrice":"70986","liqPrice":"","riskLimitValue":"2000000","takeProfit":"","positionValue":"70.986","isReduceOnly":false,"tpslMode":"Full","riskId":1,"trailingStop":"0","unrealisedPnl":"-0.1281","markPrice":"71114.1","adlRankIndicator":2,"cumRealisedPnl":"-0.12302366","positionMM":"0.39787653","createdTime":"1717669747543","positionIdx":0,"positionIM":"7.14154653","seq":457843443,"updatedTime":"1717669802036","side":"Sell","bustPrice":"","positionBalance":"0","leverageSysUpdatedTime":"","curRealisedPnl":"-0.0390423","size":"0.001","positionStatus":"Normal","mmrSysUpdatedTime":"","stopLoss":"","tradeMode":0,"sessionAvgPrice":""}]},"retExtInfo":{},"time":1717678368448}
Position List Elapsed Time : 0:00:00.196680


[{'symbol': 'BTCUSDT',
  'leverage': '10',
  'autoAddMargin': 0,
  'avgPrice': '70986',
  'liqPrice': '',
  'riskLimitValue': '2000000',
  'takeProfit': '',
  'positionValue': '70.986',
  'isReduceOnly': False,
  'tpslMode': 'Full',
  'riskId': 1,
  'trailingStop': '0',
  'unrealisedPnl': '-0.1281',
  'markPrice': '71114.1',
  'adlRankIndicator': 2,
  'cumRealisedPnl': '-0.12302366',
  'positionMM': '0.39787653',
  'createdTime': '1717669747543',
  'positionIdx': 0,
  'positionIM': '7.14154653',
  'seq': 457843443,
  'updatedTime': '1717669802036',
  'side': 'Sell',
  'bustPrice': '',
  'positionBalance': '0',
  'leverageSysUpdatedTime': '',
  'curRealisedPnl': '-0.0390423',
  'size': '0.001',
  'positionStatus': 'Normal',
  'mmrSysUpdatedTime': '',
  'stopLoss': '',
  'tradeMode': 0,
  'sessionAvgPrice': ''}]

In [9]:
'''
#Get unfilled Orders
endpoint="/v5/order/realtime"
method="GET"
params='category=linear&settleCoin=USDT'
HTTP_Request(endpoint,method,params,"UnFilled")


#Cancel Order
endpoint="/v5/order/cancel"
method="POST"
params='{"category":"linear","symbol": "BTCUSDT","orderLinkId": "'+orderLinkId+'"}'
HTTP_Request(endpoint,method,params,"Cancel")
'''

'\n#Get unfilled Orders\nendpoint="/v5/order/realtime"\nmethod="GET"\nparams=\'category=linear&settleCoin=USDT\'\nHTTP_Request(endpoint,method,params,"UnFilled")\n\n\n#Cancel Order\nendpoint="/v5/order/cancel"\nmethod="POST"\nparams=\'{"category":"linear","symbol": "BTCUSDT","orderLinkId": "\'+orderLinkId+\'"}\'\nHTTP_Request(endpoint,method,params,"Cancel")\n'

In [10]:
x = [{'symbol': 'BTCUSDT',
  'leverage': '10',
  'autoAddMargin': 0,
  'stopLoss': '',
  'tradeMode': 0,
  'sessionAvgPrice': ''},
     {'symbol': 'BTCUSDT',
  'leverage': '10',
  'autoAddMargin': 0,
  'stopLoss': '',
  'tradeMode': 0,
  'sessionAvgPrice': ''}
    ]
x_ = [entry["symbol"] for entry in x]
len(x)

2

In [11]:
active_symbol = ["ETH","BTC"]
active_symbol_ = ["CTC"]
symbol = "ETH"

symbol not in active_symbol+active_symbol_

False

In [12]:
side = "up"
side = "Buy" if side == "up" else "Sell"
side

'Buy'

In [33]:
# DEMO NET
api_key = "GMUtVbMzRhohkOagoW"
secret_key = "vexur1jjKy5AiQE0rXMhVYbjRFRy6jVxhIhf"
side = "up"
entry = str(60000)
take_profit = entry
stop_loss = entry


def active_orders(api_key,api_secret):
    endpoint="/v5/order/realtime"
    method="GET"
    params='category=linear&settleCoin=USDT'
    res = HTTP_Request(endpoint,method,params,"Position List")
    res = res.json()
    num_orders = len(res['result']['list'])
    symbols = [entry["symbol"] for entry in res['result']['list']]
    return num_orders,symbols


def active_positions(api_key,api_secret):
    endpoint="/v5/position/list"
    method="GET"
    params='category=linear&settleCoin=USDT'
    res = HTTP_Request(endpoint,method,params,"Position List")
    res = res.json()
    num_orders = len(res['result']['list'])
    symbols = [entry["symbol"] for entry in res['result']['list']]
    return num_orders,symbols

def place_order(api_key,api_secret,side,entry,take_profit,stop_loss):
    '''
    places an order at the bybit exchange only if there is no active order or position
    '''
    endpoint="/v5/order/create"
    method="POST"
    symbol = "BTCUSDT"
    side = "Buy" if side == "up" else "Sell"
    
    params={
    "category":"linear",
    "symbol": symbol,
    "side": side,
    "orderType": "Limit",
    "qty": "0.001",
    "price": str(entry),
    "takeProfit" : str(take_profit),
    "tpLimitPrice" : str(take_profit),
    "tpOrderType" : "Limit",
    "stopLoss" : str(stop_loss),
    "slLimitPrice" : str(stop_loss),
    "slOrderType" : "Limit",
    "tpslMode" : "Partial"
    }
    params = json.dumps(params)
    
    # check active order
    active_positions_count, active_symbols_pos = active_positions(api_key, api_secret)
    active_orders_count, active_symbols_order = active_orders(api_key, api_secret)

    if symbol not in active_symbols_pos + active_symbols_order:
        if (active_positions_count < 1) & (active_orders_count < 1):
            res = HTTP_Request(endpoint, method, params, "Create Order")
        else: 
            res = "No order placed because of active orders or positions."
    else:
        res = "No order placed because of active Symbols."
            
    return res
    
place_order(api_key,secret_key,side,entry,take_profit,stop_loss)

<Response [200]>
{"retCode":0,"retMsg":"OK","result":{"nextPageCursor":"","category":"linear","list":[]},"retExtInfo":{},"time":1717682363670}
Position List Elapsed Time : 0:00:00.565519
<Response [200]>
{"retCode":0,"retMsg":"OK","result":{"nextPageCursor":"","category":"linear","list":[]},"retExtInfo":{},"time":1717682364382}
Position List Elapsed Time : 0:00:00.707034
<Response [200]>
{"retCode":0,"retMsg":"OK","result":{"orderId":"791d5144-fd96-42f3-9acd-790c33ded61d","orderLinkId":""},"retExtInfo":{},"time":1717682364910}
Create Order Elapsed Time : 0:00:00.524859


<Response [200]>

# fetch kline

In [ ]:

def fetch_kline():
    '''
    fetches data from bybit with hard-coded characteristics. 
    '''
    # Define the endpoint and parameters
    base_url = "https://api-demo.bybit.com"
    endpoint = "/v5/market/kline"
    symbol = "BTCUSDT"  # Example trading pair
    interval = "15"    # Kline interval, e.g., "1", "5", "15", "60", "240", etc.
    limit = 10        # Number of klines to fetch
    start_time = int(time.time()) - 3600 * 24  # Start time in seconds (e.g., 24 hours ago)
    
    # Construct the URL
    url = f"{base_url}{endpoint}?category=linear&symbol={symbol}&interval={interval}&limit={limit}"#&start_time={start_time}"
    
    # Make the GET request
    response = requests.get(url,headers={}, data={})
    data = response.json() 
    
    # print(data)
    # print(Timestamp(data["time"],unit="ms"))
    return data
fetch_kline()

# pybit functions

In [ ]:
import time
import hmac
import hashlib
from Crypto.Hash import SHA256
from Crypto.PublicKey import RSA
from Crypto.Signature import PKCS1_v1_5

def generate_signature(use_rsa_authentication, secret, param_str):
    def generate_hmac():
        hash = hmac.new(
            bytes(secret, "utf-8"),
            param_str.encode("utf-8"),
            hashlib.sha256,
        )
        return hash.hexdigest()

    def generate_rsa():
        hash = SHA256.new(param_str.encode("utf-8"))
        encoded_signature = base64.b64encode(
            PKCS1_v1_5.new(RSA.importKey(secret)).sign(
                hash
            )
        )
        return encoded_signature.decode()

    if not use_rsa_authentication:
        return generate_hmac()
    else:
        return generate_rsa()

In [ ]:
# pybit functions: 


parameters = {
    'category' : 'inverse',
    'symbol' : 'BTCUSDT',
    'side': 'Buy',
    'orderType': 'Limit',
    'qty': '0.003',
    'price': '60000'
}

timestamp = str(int(time.time() * 1000))

payload = "&".join(
            [
                str(k) + "=" + str(v)
                for k, v in sorted(parameters.items())
                if v is not None
            ])

param_str = str(timestamp) + api_key + str(recv_window) + payload

generate_signature(rsa_authentication, api_secret, param_str)


def auth(self, payload, recv_window, timestamp):
    """
    Prepares authentication signature per Bybit API specifications.
    """

    if api_key is None or api_secret is None:
        raise PermissionError("Authenticated endpoints require keys.")

    param_str = str(timestamp) + api_key + str(recv_window) + payload

    return generate_signature(
        rsa_authentication, api_secret, param_str
    )
    
signature = auth(
                payload=req_params,
                recv_window=recv_window,
                timestamp=timestamp,
            )


# first tries and other stuff

In [ ]:
import time
import hashlib
import hmac
import requests

# Replace these with your API key and secret
# api_key_testnet = "yeOt7DrchkqM9r7sQy"
# api_secret_testnet = "JJHVxNX2wc7di918pwjxSJemJd6j1bsAVRmO"

# hmac api keys
api_key = "vgcrg9nWw7owVYEDdc"
api_secret = "YPz7eb0YzhOpa9s2yMEj3P4UdXrmSEtqRDCP"


def sign(secret,payload):
    return hmac.new(bytes(secret, 'utf-8'), bytes(payload, 'utf-8'), hashlib.sha256).hexdigest()

def place_order(api_key, api_secret, side, price,take_profit,stop_loss):
    
    url = 'https://api-testnet.bybit.com/v5/order/create'

    params = {
        #'timestamp': int(time.time() * 1000),
        #'api_key': api_key,
        #'recv_window': 5000,
        'category' : 'inverse',
        'symbol' : 'BTCUSDT',
        #'isLeverage' : '0',
        'side': side,
        'orderType': 'Limit',
        'qty': '0.003',
        'price': price,
        #'time_in_force': 'GTC',
        #'takeProfit' : take_profit,
        #'tpLimitPrice' : take_profit,
        #'tpOrderType' : 'Limit',
        #'stopLoss' : stop_loss,
        #'slLimitPrice' : stop_loss,
        #'slOrderType' : 'Limit',
        #'tpslMode' : 'Partial',
    }

    # Create the query string
    query_string = '&'.join([f"{key}={value}" for key, value in params.items()])

    print(params.items())
    ts = str(int(time.time() * 1000))
    
    # Sign the query string
    params['sign'] = sign(ts, api_secret,query_string)

    # Send the request
    response = requests.post(url, data=params)
    
    return response

# Example usage:
symbol = 'BTCUSDT'
side = 'Buy'
price = 70000  # Limit price
take_profit = 71000
stop_loss = 68000


order_response = place_order(api_key, api_secret, side, price,take_profit,stop_loss)

print(order_response)
print(order_response.json())


In [ ]:
import requests

url = "https://api-testnet.bybit.com/v5/order/create"
timestamp = int(time.time() * 1000)

payload = "{\n  \"category\": \"linear\",\n  \"symbol\": \"BTCUSDT\",\n  \"side\": \"Buy\",\n  \"orderType\": \"Limit\",\n  \"qty\": \"0.002\",\n  \"price\": \"70000\",\n  \"triggerPrice\": null,\n  \"triggerDirection\": null,\n  \"triggerBy\": null,\n  \"orderFilter\": null,\n  \"orderIv\": null,\n  \"timeInForce\": \"GTC\",\n  \"positionIdx\": 0,\n  \"orderLinkId\": \"test-xx1\",\n  \"takeProfit\": null,\n  \"stopLoss\": null,\n  \"tpTriggerBy\": null,\n  \"slTriggerBy\": null,\n  \"reduceOnly\": false,\n  \"closeOnTrigger\": false,\n  \"smpType\": null,\n  \"mmp\": null,\n  \"tpslMode\": null,\n  \"tpLimitPrice\": null,\n  \"slLimitPrice\": null,\n  \"tpOrderType\": null,\n  \"slOrderType\": null\n}"
headers = {
    "Content-Type": "application/json",
    'X-BAPI-API-KEY': str(api_key_testnet),
    'X-BAPI-TIMESTAMP': str(timestamp),
    'X-BAPI-RECV-WINDOW': '20000',
    'X-BAPI-SIGN': '8b96a595c486b7b5ed0d7ad66f001b4d206026e761d73406080f9cae57cd9ecd'
}

  headers = {
                    "Content-Type": "application/json",
                    "X-BAPI-API-KEY": api_key,
                    "X-BAPI-SIGN": signature,
                    "X-BAPI-SIGN-TYPE": "2",
                    "X-BAPI-TIMESTAMP": str(timestamp),
                    "X-BAPI-RECV-WINDOW": str(recv_window),
                }

response = requests.request("POST", url, headers=headers, data=payload)

print(response.text)